#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [91]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

1. 뉴스기사 선언

In [92]:
article = """
원하는 이미지를 뚝딱 만들어주는 생성형 인공지능(AI)이 날로 정교해지고 있지만 오남용을 막기 위한 ‘안전장치’는 제각각이다. 일론 머스크 테슬라 최고경영자(CEO)의 AI 스타트업 xAI가 출시한 이미지 생성 AI 챗봇이 논란의 중심에 섰다. 이미지 생성에 대한 제한이 거의 없어 가짜 이미지 확산 우려가 제기되고 있다.

17일(현지시간) 정보기술(IT) 전문 매체 벤처비트 등에 따르면 구글은 지난 15일 자사 이미지 생성 AI 모델의 최신 버전 ‘이마젠3’를 AI 도구 ‘이미지FX’를 통해 미국 사용자에게 선보였다. 지난 13일에는 xAI가 이미지 생성 기능을 추가한 신형 AI 챗봇 ‘그록2’를 시험 버전으로 출시했다. 그록2에는 독일 스타트업 블랙포레스트랩스의 이미지 생성 AI 모델을 적용했다. X(옛 트위터)의 유료 구독형 서비스 이용자만 이용 가능하다. 이들 서비스는 오픈AI의 달리, 미드저니 등 쟁쟁한 이미지 생성 AI와 경쟁한다.

서비스 이용 방식은 대부분 비슷하다. 텍스트로 어떤 이미지를 만들어달라고 입력하면, 그에 걸맞은 결과물을 내놓는다. 기업들은 자사 AI가 이용자의 요청을 잘 이해해 매우 사실적이고 자연스러운 이미지를 생성한다고 홍보한다.

눈에 띄는 차이를 만드는 건 기술 혁신과 책임 사이의 균형이다. 구글은 “이마젠3는 데이터·모델 개발에서 생산에 이르기까지 당사의 최신 안전 및 책임 혁신을 통해 구축됐다”고 강조하고 있다. 생성된 이미지에는 눈에는 보이지 않는 디지털 워터마크가 표시된다. 미국 민주당 대선 후보인 카멀라 해리스 부통령, 인기 가수 테일러 스위프트 등 유명 인사들의 이미지는 생성하지 않는다. 정도의 차이가 존재하지만 달리, 미드저니 등도 안전장치를 두고 있다. 다만 이미지 생성 제한을 두고 있어도 요청을 조금만 다른 방식으로 하면 이를 우회할 가능성은 여전히 있다.

하지만 그록2는 제한 자체를 거의 두지 않은 것으로 나타났다. 임신한 해리스 부통령의 배를 만지는 공화당 대선 후보 도널드 트럼프 전 대통령, 마약을 흡입하는 빌 게이츠 마이크로소프트(MS) 창업자, 속옷만 입은 스위프트 같은 ‘가짜 이미지’를 만들었다는 사례가 엑스와 언론을 통해 전해졌다. 담배 피우는 미키마우스처럼 저작권이 있는 캐릭터 이미지도 생성한다. IT 전문 매체 더버지는 “우리의 테스트에서 그록은 ‘벌거벗은 여성 이미지를 생성해달라’는 단 하나의 요청을 거부했다”고 전했다.

머스크는 지난 15일 엑스에 “그록은 세상에서 가장 재미있는 AI”라고 올렸다. 2022년 트위터를 인수한 그는 ‘표현의 자유’를 방패 삼아 가짜뉴스, 혐오표현 등 유해 콘텐츠를 방치하고 있다는 비판을 받아 왔다. 그록2의 안전장치를 최소화한 것도 이 같은 행보의 연장선으로 볼 수 있다. 엑스에 게시되는 불쾌한 콘텐츠에 반발한 광고주가 이탈할 가능성이 거론된다. 반면 제한 없는 이미지 생성 서비스가 더 많은 이용자를 끌어모을 수도 있다.

미국에선 대선을 앞두고 있는 만큼 후보들의 가짜 이미지 확산에 따른 우려가 커지는 상황이다. 경쟁사와는 다른 노선을 택한 그록2의 등장은 AI 규제 논의를 촉진할 것으로 전망된다.
"""

2. 텍스트 전처리

In [93]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 이미지 뚝딱 생 성형 인공 지능 AI 이 날로 남용 위 안전 장치 제각각 론 머스크 테슬라 최고경영자 CEO 의 AI 스타트업 xAI 출시 이미지 생 AI 챗봇 논란 중심 이미지 생 대한 제한 거의 가짜 이미지 확산 우려 제기 현지 시간 정보기술 IT 전문 매체 처 비트 등 구글 지난 자사 이미지 생 AI 모델 최신 버전 젠 를 AI 도구 이미지 FX 를 통해 미국 사용자 지난 xAI 이미지 생 기능 추가 신형 AI 챗봇 그록 를 시험 버전 출시 그록 독일 스타트업 블랙 포레 스트랩 스 이미지 생 AI 모델 적용 X 트위터 의 유료 구독 서비스 이용자 이용 이 서비스 오픈 AI 의 달리 미드 저니 등 쟁쟁 이미지 생 AI 경쟁 서비스 이용 방식 대부분 텍스트 이미지 달라 입력 그 걸 결과물 기업 자사 AI 이용자 요청 이해해 매우 사실 이미지 생 홍보 눈 차이 건 기술 혁신 책임 사이 균형 구글 젠 데이터 모델 개발 생산 당사 최신 안전 및 책임 혁신 통해 구축 고 강조 생 이미지 눈 보이지 디지털 워터마크 표시 미국 민주당 대선 후보 카멀 해리스 부통령 인기 가수 테일러 스위프트 등 유명 인사 이미지 생 정도 차이 존재 달리 미드 저니 등 안전 장치 다만 이미지 생 제한 요청 조금 다른 방식 우회 가능성 그록 제한 자체 거의 것 임신 해리스 부통령 배 공화당 대선 후보 도널드 트럼프 전 대통령 마약 흡입 게이츠 마이크로소프트 MS 창업 속옷 스위프트 가짜 이미지 를 사례 엑스 언론 통해 담배 미키마우스 저작권 캐릭터 이미지 생 IT 전문 매체 더버 우리 테스트 그록 여성 이미지 생 단 하나 요청 거부 고 전 머스크 지난 엑스 그록 세상 가장 AI 트위터 인수 그 표현 자유 를 방패 가짜 뉴스 혐오 표현 등 유해 콘텐츠 방치 비판 그록 의 안전 장치 최소 것 이 행보 장선 볼 수 엑스 게시 콘텐츠 반발 광고주 이탈 가능성 거론 반면 제한 이미지 생 서비스 더 이용자 수도 미국 대선 만큼 후보 가짜 이미지 확산 우려 상황 경쟁 사 다른 

3. N-Gram 생성 및 SBERT

In [94]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 422
N-Gram 다섯개 출력 : ['ai' 'ai 경쟁' 'ai 규제' 'ai 날로' 'ai 달리']
SBERT 분석 후 뉴스 기사 : [[-4.02229279e-02  4.39844400e-01 -2.52229124e-01 -1.03217416e-01
  -8.05686936e-02  2.78854296e-02  6.79936469e-01 -5.00642844e-02
   5.99704325e-01  4.92594093e-02 -1.88823745e-01 -6.81530476e-01
  -6.81208134e-01 -3.15475613e-01 -8.17924619e-01  9.21631232e-02
  -3.32386613e-01  2.86498517e-02 -2.59971227e-02  1.67055741e-01
  -4.74073887e-01 -8.81066680e-01  1.90467238e-01 -3.05593550e-01
   2.46903241e-01  3.74673814e-01  4.52195287e-01  1.05546665e+00
  -3.59695703e-02  3.97763699e-01 -2.07238749e-01 -1.92989439e-01
  -5.05457520e-01 -3.46564591e-01 -6.13152146e-01  9.86369848e-01
  -1.95343614e-01  6.32042736e-02  1.87855914e-01  9.06208515e-01
  -5.72364688e-01 -7.32039452e-01  6.13846704e-02  6.49781823e-02
   5.02940640e-02 -1.32345967e-02 -1.53065652e-01  7.36511111e-01
  -1.21761389e-01  1.00635314e+00 -2.59802461e-01  7.64706552e-01
  -4.53794241e-01 -4.21926647e-01 -7.99321756e-02  2

4. Max Sum Similarity

In [95]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['남용 안전', '챗봇', '가짜 뉴스', '혁신 책임', '쟁쟁 이미지']


5. 테마 가져오기

In [96]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [97]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
# print(f'뉴스 기사 : \n{article}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {cosine_recommend[-i]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1


뉴스 기사와 연관 되어 있는 테마 5개
1. 사물인터넷 : 사물인터넷
2. 지능형로봇/인공지능(AI) : 지능형로봇/인공지능(AI)
3. AI 챗봇(챗GPT 등) : AI 챗봇(챗GPT 등)
4. 로봇(산업용/협동로봇 등) : 로봇(산업용/협동로봇 등)
5. 인터넷 대표주 : 인터넷 대표주
